In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Embedding, Conv1D, GlobalMaxPooling1D,
                                     Concatenate, Dense, Dropout, SpatialDropout1D)
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# 1. Custom Utility Functions
# -----------------------------
def custom_train_test_split(X, y, test_size=0.2, random_state=42):
    """A simple train/validation split using numpy."""
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split = int(X.shape[0] * test_size)
    test_idx = indices[:split]
    train_idx = indices[split:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

def custom_f1_score(y_true, y_pred):
    """
    Compute the F1 score for binary classification.
    Both y_true and y_pred should be numpy arrays.
    """
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    f1 = 2 * precision * recall / (precision + recall + 1e-7)
    return f1

# -----------------------------
# 2. Load the Data
# -----------------------------
# Load training data (with labels)
train_df = pd.read_csv('/content/train.csv')
# Load test data (without labels)
test_df = pd.read_csv('/content/test.csv')

# Extract texts and labels from training data
texts_train = train_df['text'].astype(str).values
labels_train = train_df['target'].values

# Extract texts from test data
texts_test = test_df['text'].astype(str).values

# -----------------------------
# 3. Preprocess the Data (Tokenization & Padding)
# -----------------------------
max_words = 20000    # Maximum number of words to consider
max_length = 100     # Maximum tweet length (in tokens)
embedding_dim = 128  # Increase embedding dimension for better expressiveness

# Create and fit the tokenizer on the training texts
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts_train)

# Convert texts to sequences for both train and test data
train_sequences = tokenizer.texts_to_sequences(texts_train)
test_sequences = tokenizer.texts_to_sequences(texts_test)

# Pad sequences to ensure uniform length
X_train_full = pad_sequences(train_sequences, maxlen=max_length)
X_test = pad_sequences(test_sequences, maxlen=max_length)

# -----------------------------
# 4. Split Training Data into Training and Validation Sets
# -----------------------------
X_train, X_val, y_train, y_val = custom_train_test_split(X_train_full, labels_train, test_size=0.2, random_state=42)

# -----------------------------
# 5. Build the Improved Multi-Kernel CNN Model
# -----------------------------
input_layer = Input(shape=(max_length,), name='input')

# Trainable Embedding layer with random initialization
embedding_layer = Embedding(input_dim=max_words,
                            output_dim=embedding_dim,
                            input_length=max_length,
                            name='embedding')(input_layer)

# Apply SpatialDropout1D to regularize embeddings
x = SpatialDropout1D(0.2)(embedding_layer)

# Create parallel convolutional layers with different kernel sizes
filter_sizes = [3, 4, 5]
conv_layers = []
for size in filter_sizes:
    conv = Conv1D(filters=128, kernel_size=size, activation='relu')(x)
    pool = GlobalMaxPooling1D()(conv)
    conv_layers.append(pool)

# Concatenate the pooled features
concatenated = Concatenate()(conv_layers)

# Add dropout for further regularization
x = Dropout(0.5)(concatenated)

# Fully connected layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Final output layer with sigmoid activation for binary classification
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# -----------------------------
# 6. Train the Model with EarlyStopping
# -----------------------------
batch_size = 32
epochs = 20

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# -----------------------------
# 7. Evaluate the Model on the Validation Set
# -----------------------------
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)
f1 = custom_f1_score(np.array(y_val), y_val_pred)
print("Validation F1 Score: {:.4f}".format(f1))

# -----------------------------
# 8. Generate Predictions on the Test Set and Create a Submission File
# -----------------------------
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int).flatten()

# If your test file contains an 'id' column, include it in the submission.
if 'id' in test_df.columns:
    submission = pd.DataFrame({'id': test_df['id'], 'target': y_test_pred})
else:
    submission = pd.DataFrame({'target': y_test_pred})

submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 100, 128)       │      2,560,000 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spatial_dropout1d_1       │ (None, 100, 128)       │              0 │ embedding[0][0]        │
│ (SpatialDropout1D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 98, 128)        │         49,280 │ spatial_dropout1d_1[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 97, 128)        │         65,664 │ spatial_dropout1d_1[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 96, 128)        │         82,048 │ spatial_dropout1d_1[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_3    │ (None, 128)            │              0 │ conv1d_3[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_4    │ (None, 128)            │              0 │ conv1d_4[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_5    │ (None, 128)            │              0 │ conv1d_5[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 384)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 384)            │              0 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         49,280 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            129 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,806,401 (10.71 MB)

 Trainable params: 2,806,401 (10.71 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
191/191 ━━━━━━━━━━━━━━━━━━━━ 23s 104ms/step - accuracy: 0.5942 - loss: 0.6599 - val_accuracy: 0.7878 - val_loss: 0.4723
Epoch 2/20
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.8489 - loss: 0.3737 - val_accuracy: 0.8022 - val_loss: 0.4530
Epoch 3/20
191/191 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.9347 - loss: 0.1953 - val_accuracy: 0.7845 - val_loss: 0.5182
Epoch 4/20
191/191 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.9722 - loss: 0.1015 - val_accuracy: 0.7773 - val_loss: 0.6701
Epoch 5/20
191/191 ━━━━━━━━━━━━━━━━━━━━ 40s 105ms/step - accuracy: 0.9856 - loss: 0.0498 - val_accuracy: 0.7674 - val_loss: 0.7164
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step
Validation F1 Score: 0.7578
102/102 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
Submission file saved as 'submission.csv'.
